## Topic model 

### Twitter data for  Delta, Southwest, American, Jetblue Airlines 

#### Imports

In [103]:
# Importing modules
import pandas as pd
import os

os.chdir('..')

# Cobining airline tweet data 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#/Users/abdulaleemseyed/Desktop/DS_Projects/NLP

#### Loading data

In [104]:
# Cobining airline twitter tweet data

Delta = pd.read_csv(DeltaAirlines_tweets.csv")
Southwest = pd.read_csv("SouthWest_tweets.csv")
American = pd.read_csv("AmericanAirlines_tweets.csv")
Jetblue = pd.read_csv("NLP/JetBlue_tweets.csv")

airtweets = pd.concat([Delta, Southwest, American, Jetblue], sort=False)
#aritweets = pd.drop
airtweets.head(5)

,geo,text,user,location,country,entities,sentiment,neg,neu,pos,compound,Unnamed: 0
0,NaN,"The guide on #Delta Airlines #cancellationpolicy will help you to find the #Refund status on #tickets, risk-free…",flydealfare,United States,United States,"['', ('Delta Airlines', 'ORG'), '', ('Refund', 'ORG'), '']","{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'compound': 0.4019}",0.0,0.856,0.144,0.4019,NaN
1,NaN,Hand sanitizer readily available.#Airlines #AirlineBailout #deltaairlines #Delta #UnitedAirlines #AmericanAirlines #flying,greatisenough,"Greensboro, NC",USA,"['', '']","{'neg': 0.0, 'neu': 0.738, 'pos': 0.262, 'compound': 0.4939}",0.0,0.738,0.262,0.4939,NaN
2,NaN,#Delta has been flying five daily flights out of Love Field to Atlanta for years on a Southwest Airlines gate. …,AtlBizRetweet,United States,United States,"['', ('Delta', 'ORG'), ('five', 'CARDINAL'), ('daily', 'DATE'), ('Love Field', 'GPE'), ('Atlanta', 'GPE'), ('years', 'DATE'), ('Southwest Airlines', 'ORG')]","{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.6369}",0.0,0.811,0.189,0.6369,NaN
3,NaN,#Delta has been flying five daily flights out of Love Field to Atlanta for years on a Southwest Airlines gate.,AtlBizChron,"Atlanta, Georgia",USA,"[('Delta', 'ORG'), ('five', 'CARDINAL'), ('daily', 'DATE'), ('Love Field', 'GPE'), ('Atlanta', 'GPE'), ('years', 'DATE'), ('Southwest Airlines', 'ORG')]","{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.6369}",0.0,0.811,0.189,0.6369,NaN
4,NaN,"So the airline industry gets free money from the government without the government getting a stake in them, but the…",wallstreet92,Michigan,USA,[],"{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'compound': 0.2846}",0.0,0.893,0.107,0.2846,NaN


#### Data Cleaning 

In [67]:
airtweets = airtweets['text']

In [69]:
# Load the regular expression library
import re
airtweets.head()

0            The guide on #Delta Airlines #cancellationpolicy will help you to find the #Refund status on #tickets, risk-free… 
1    Hand sanitizer readily available.#Airlines #AirlineBailout #deltaairlines #Delta #UnitedAirlines #AmericanAirlines #flying
2              #Delta has been flying five daily flights out of Love Field to Atlanta for years on a Southwest Airlines gate. …
3               #Delta has been flying five daily flights out of Love Field to Atlanta for years on a Southwest Airlines gate. 
4         So the airline industry gets free money from the government without the government getting a stake in them, but the… 
Name: text, dtype: object

In [73]:
# Load the regular expression library
import re

# Convert the titles to lowercase

(1261,)

#### Tokenize words and further clean-up text

In [75]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = airtweets.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['the', 'guide', 'on', 'delta', 'airlines', 'will', 'help', 'you', 'to', 'find', 'the', 'refund', 'status', 'on', 'tickets', 'risk', 'free']


#### Creating Bigram and Trigram Models

In [76]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

In [77]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdulaleemseyed/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [78]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [79]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['guide', 'delta', 'airline', 'help', 'find', 'refund', 'status', 'ticket', 'risk', 'free']


#### Data transformation: Corpus and Dictionary
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [80]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


#### Hyperparameter tuning

In [86]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

#### Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [84]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }


# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|          | 0/270 [00:00<?, ?it/s]

NameError: name 'CoherenceModel' is not defined

In [99]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [100]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"flight" + 0.014*"jetblue" + 0.010*"airline" + 0.010*"american" + '
  '0.008*"call" + 0.008*"attendant" + 0.007*"male" + 0.007*"proceed" + '
  '0.007*"bl" + 0.007*"calls_black"'),
 (1,
  '0.039*"choo" + 0.031*"delta" + 0.030*"airline" + 0.022*"state" + '
  '0.022*"southwest" + 0.020*"cost" + 0.020*"california" + 0.020*"speed" + '
  '0.020*"could_purchase" + 0.008*"go"'),
 (2,
  '0.054*"airline" + 0.049*"free" + 0.048*"cash" + 0.048*"spend" + 0.048*"buy" '
  '+ 0.047*"last" + 0.047*"group" + 0.047*"big" + 0.047*"share" + '
  '0.046*"flow"'),
 (3,
  '0.017*"airline" + 0.015*"delta" + 0.012*"get" + 0.007*"be" + '
  '0.006*"buffett" + 0.006*"expert" + 0.006*"bailoutsall_know" + 0.006*"ba" + '
  '0.005*"jetblue" + 0.004*"united"'),
 (4,
  '0.022*"flight" + 0.013*"hour" + 0.013*"bailout" + 0.012*"airline" + '
  '0.012*"refund" + 0.011*"part" + 0.011*"jfk_awaite" + 0.011*"delayed_austin" '
  '+ 0.011*"understandable" + 0.011*"journey_home"'),
 (5,
  '0.041*"airline" + 0.027*"ame

In [101]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

/Users/abdulaleemseyed/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.221331 -0.004989       1        1  24.815140
5     -0.038453 -0.041558       2        1  22.619328
4     -0.066505 -0.044742       3        1  16.632954
1     -0.039226  0.106306       4        1  14.364415
0     -0.041365 -0.029261       5        1  12.137823
3     -0.035782  0.014244       6        1   9.430340, topic_info=    Category        Freq       Term       Total  loglift  logprob
304  Default  135.000000      spend  135.000000  30.0000  30.0000
300  Default  136.000000       cash  136.000000  29.0000  29.0000
302  Default  132.000000      group  132.000000  28.0000  28.0000
423  Default  130.000000      share  130.000000  27.0000  27.0000
422  Default  129.000000       flow  129.000000  26.0000  26.0000
..       ...         ...        ...         ...      ...      ...
20    Topic6    3.250156     flight  135.652420  -1.3702  -5.7727
231   Topic6    2.727486     change   31.115871  -0.0731  -5.9480
28    Topic6    2.483714      money   16.273556   0.4815  -6.0416
233   Topic6    2.385046        fee   14.369825   0.5653  -6.0822
23    Topic6    2.372135  southwest   47.065838  -0.6265  -6.0876

[322 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
670       1  0.503141     aal
670       2  0.201256     aal
670       3  0.100628     aal
1379      1  0.112823  airbus
1379      2  0.676935  airbus
...     ...       ...     ...
362       6  0.044447   would
24        1  0.057446    year
24        2  0.574465    year
24        3  0.229786    year
24        5  0.114893    year

[876 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 5, 2, 1, 4])